Test comparison of two scalars using SEAL API from tenseal module. Parameters and setup is explained in 5_ckks_basics on the SEAL github page. 

In [238]:
import tenseal.sealapi as seal
import util
import compare_util as cu
from math import log2,ceil
import ctypes

In [261]:
parms = seal.EncryptionParameters(seal.SCHEME_TYPE.CKKS)
poly_modulus_degree = 8192*2
parms.set_poly_modulus_degree(poly_modulus_degree)
parms.set_coeff_modulus(seal.CoeffModulus.Create(poly_modulus_degree, [60, 40,40,40,40,40, 60]))
scale = pow(2.0, 40)

context = seal.SEALContext(parms,True,seal.SEC_LEVEL_TYPE.TC128)

util.print_parameters(context)

print("|\t Max Bit Count: " + str(seal.CoeffModulus.MaxBitCount(poly_modulus_degree, seal.SEC_LEVEL_TYPE.TC128)))

/
|Encryption parameters: 
|	scheme: CKKS
|	poly_modulus_degree: 16384
|	coeff_modulus_size: 320 (60 40 40 40 40 40 60) bits
|	 Max Bit Count: 438


Generate keys using SEAL keygenerator. Generate encoder, evaluator, encryptor and decryptor. 

In [262]:

keygen = seal.KeyGenerator(context)
secret_key = keygen.secret_key()

public_key = seal.PublicKey()
keygen.create_public_key(public_key)

relin_keys = seal.RelinKeys()
keygen.create_relin_keys(relin_keys)

galois_keys = seal.GaloisKeys()
keygen.create_galois_keys()

encryptor = seal.Encryptor(context, public_key)
evaluator = seal.Evaluator(context)
decryptor = seal.Decryptor(context, secret_key)

encoder = seal.CKKSEncoder(context)
slot_count = seal.CKKSEncoder.slot_count(encoder)

print('Number of slots: ' + str(slot_count))

Number of slots: 8192


Let $x$ be the encrypted difference between two number in $a,b\in[0,1]$, such that $x\in[-1,1]$.

In [263]:
a=0.2;b=0.8
x = a-b
x_plain = seal.Plaintext()
encoder.encode(x,scale, x_plain)

x_enc = seal.Ciphertext()
encryptor.encrypt(x_plain, x_enc)

In [294]:
# Worst way of copying a ciphertext - multiply by plaintext 1. Consumes one modulus level. 
def bad_copy(cipher : seal.Ciphertext,scale : int, evaluator  : seal.Evaluator, encoder : seal.CKKSEncoder):
    dummy_plain = seal.Plaintext()
    encoder.encode(1, scale, dummy_plain)

    copy = seal.Ciphertext()
    evaluator.multiply_plain(cipher, dummy_plain, copy)
    # Force scale to be the same as original. Introduces error
    evaluator.rescale_to_next_inplace(copy)
    return copy 


Polynomial approximations of the comparison function using polynomials f_3(x), g_3(x) from "Efficient Sorting of Homomorphic Encrypted Data with k-way Sorting Network". Note that the argument to the function in this case will be the encrypted `difference`between $a$ and $b$, i.e., $a-b$. 

* g_3(x) = (35x-35x^3+21x^5-5x^7)/2^4

* f_3(x) = (4589x-16577x^3+25614x^5-12860x^7)/2^10

Comparison function is implemented in the paper as

* (x>y) := $(f_3^{(d_f)}\circ g_3^{(d_g)}(x-y) + 1)/2$

$d_g$ and $d_f$ are not specified in the paper. Will begin testing with $d_f=d_g=1$. 

Note that the highest degree term is $x^7 = (x^2)^2(x^2*)x$, requiring four multiplications. We will begin by calculating the different temrs, 
- $x^7$
- $x^5$
- $x^3$

In [302]:
# A function for controlling size and scale of ciphertext after multiplication
def relinearize_and_rescale_inplace(cipher : seal.Ciphertext, eval : seal.Evaluator, relin_keys : seal.RelinKeys):
    # Control size by relinearization
    evaluator.relinearize_inplace(cipher, relin_keys)
    # Control scale by rescaling
    evaluator.rescale_to_next_inplace(cipher)

# evaluator::exponentiate not supported for CKKS, write separate function for exponentiation, note that "res" 
# should be an encryption of 1 for this to work. 
def square_and_multiply(cipher : seal.Ciphertext, exp : int,  evaluator : seal.Evaluator, relin_keys : seal.RelinKeys,scale : int, encoder: seal.CKKSEncoder, res : seal.Ciphertext):  
    if exp==0:
        raise ValueError("Exponent cannot be zero --> transparent ciphertext.")
    binary_exp = bin(exp)[2:]
    for char in binary_exp[::-1]:
        if char=='1':
            if res.data() == None:
                raise ValueError("Ciphertext res must be initialized to 1.")
            else:
                evaluator.multiply_inplace(res, cipher)
                relinearize_and_rescale_inplace(res, evaluator, relin_keys)
                evaluator.square_inplace(cipher)
                relinearize_and_rescale_inplace(cipher, evaluator, relin_keys)
        else:
            evaluator.square_inplace(cipher)
            relinearize_and_rescale_inplace(cipher,evaluator, relin_keys)
            evaluator.mod_switch_to_next_inplace(res)


In [306]:
#Fresh ciphertext (for debugging only, until I manage to make a copy of existing ciphertext)
a=0.2;b=0.8
x = a-b
x_plain = seal.Plaintext()
encoder.encode(x,scale, x_plain)

x_enc = seal.Ciphertext()
encryptor.encrypt(x_plain, x_enc)

In [307]:
dummy_plain = seal.Plaintext()
encoder.encode(1, scale, dummy_plain)
x7_enc = seal.Ciphertext()
encryptor.encrypt(dummy_plain, x7_enc)

exponent = 4
square_and_multiply(x_enc, exponent, evaluator, relin_keys,scale, encoder, x7_enc)

plain_result = seal.Plaintext()
decryptor.decrypt(x7_enc, plain_result)
result = encoder.decode_double(plain_result)[0]
print(f"Exponent is {exponent} leading to circuit with depth {ceil(log2(exponent))}")
print(f"Encrypted result is: {result:.6E}")
print(f"True result is: {x**exponent:.6E}")
print(f"Error is: {(result-x**exponent):.4E}")


Exponent is 4 leading to circuit with depth 2
Encrypted result is: 1.296000E-01
True result is: 1.296000E-01
Error is: -3.6772E-10


In [278]:
# Test copy
x_copy = bad_copy(x_enc, scale, evaluator, encoder)

print(x_enc)
print(x_copy)

tmp_result = seal.Plaintext()
decryptor.decrypt(x_copy, tmp_result)
print(encoder.decode_double(tmp_result)[0])
decryptor.decrypt(x_enc, tmp_result)
print(encoder.decode_double(tmp_result)[0])


-0.6000000019478914
-0.6000000019478914
